Now after we have designed features related to status of questions in the dataset regardless of the actual text of the question itself, we shall design and create text-based features and then merge them with the original features in order to get the final features space.

# Importing Lbraries

In [ ]:
import spacy
!python -m spacy download en_core_web_lg
import en_core_web_lg
import numpy as np
import pandas as pd
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer

# Text-Based Features Design

Reading the original dataset as we will only use the original question1 and question2, no need for other features.

In [6]:
train_data = pd.read_csv("Data/train_data.csv", compression = "bz2")
test_data = pd.read_csv("Data/test_data.csv", compression = "bz2")

In [9]:
questions1 = (train_data["question1"]).apply(lambda x : str(x))

questions2 = (train_data["question2"]).apply(lambda x : str(x))

questions = list(questions1 + " " + questions2)

In [10]:
tfidf = TfidfVectorizer()
train_feats = tfidf.fit_transform(questions)

In [14]:
print("Each sentence has represented using vector with length =", train_feats.get_shape()[1])

Each sentence has represented using vector with length = 59480


In [23]:
 words_in_train_data = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [27]:
nlp = en_core_web_lg.load()

In [ ]:
vectors = pd.DataFrame(columns = [np.arange(300)])

i = 0

for question in questions:

  idf = 0
  vector = np.zeros(300, dtype = np.float32)
  
  for word in question.split():

     if word in words_in_train_data:
       
       word_idf = words_in_train_data[word]
       idf += words_in_train_data[word]
       vector = np.add(vector, nlp(word).vector * word_idf)
     
     else:
       vector = np.add(vector, nlp(word).vector)

  if idf != 0:
    vector = np.divide(vector, idf)
  
  vectors.loc[i] = vector
  i += 1

In [ ]:
vectors.to_pickle("train_vectors.pkl", compression = "bz2")

In [29]:
questions1 = (test_data["question1"]).apply(lambda x : str(x))

questions2 = (test_data["question2"]).apply(lambda x : str(x))

questions = list(questions1 + " " + questions2)

We here take the Idf of the word in each document in test data from train data transformed using TfIdf, we should not transformed test data, as test data here is just for testing but in real life it's not related to each other, so we can't count on it to do anyting but just use the train data as it is the source of our informations.

In [ ]:
vectors = pd.DataFrame(columns = [np.arange(300)])

i = 0

for question in questions:

  idf = 0
  vector = np.zeros(300, dtype = np.float32)
  
  for word in question.split():

     if word in words_in_train_data:
       
       word_idf = words_in_train_data[word]
       idf += words_in_train_data[word]
       vector = np.add(vector, nlp(word).vector * word_idf)
     
     else:
       vector = np.add(vector, nlp(word).vector)

  if idf != 0:
    vector = np.divide(vector, idf)
  
  vectors.loc[i] = vector
  i += 1

In [31]:
vectors.to_pickle("test_vectors.pkl", compression = "bz2")